In [14]:
from medcat.cogstack.cogstack_conn import CogStackConn
import pandas as pd
import numpy as np

## Connect to CogStack

In [15]:
INDEX = 'sample_observations_view'
HOST = '10.200.112.166'

In [16]:
conn = CogStackConn(HOST, scheme='http')

Username:
Password:········


## Get the data (records that contain text)

In [17]:
query = {
  "query": {
      "bool": {
          "must_not": {
              "term": {"encounter_document": "none"}
          },
      },
  },
  "_source": ['encounter_document', 'patient_gender', 'patient_id', 'patient_birth_date']
}

In [18]:
docs = conn.get_docs_generator(query=query, index=INDEX, es_gen_size=10000)

## Put all the data in a CSV

In [19]:
data = [['subject_id', 'text', 'dob', 'gender']]

In [20]:
for doc in docs:
    one_row = []
    one_row.append(doc['_source']['patient_id']) # subject_id
    one_row.append(doc['_source']['encounter_document']) # text
    one_row.append(doc['_source']['patient_birth_date']) # dob
    one_row.append(doc['_source']['patient_gender']) # gender
    
    data.append(one_row)

In [21]:
df = pd.DataFrame(data[1:], columns=data[0])

In [22]:
df.head()

,subject_id,text,dob,gender
0,84dd6378-2ddc-44b6-9292-2a4461bcef53,Sample Type / Medical Specialty: Consult - Hi...,1998-12-01T00:00:00.000,M
1,9929044f-1f43-4453-b2c0-a2f45dcdd4be,Sample Type / Medical Specialty: General Medi...,2014-09-23T00:00:00.000,M
2,9929044f-1f43-4453-b2c0-a2f45dcdd4be,Sample Type / Medical Specialty: General Medi...,2014-09-23T00:00:00.000,M
3,9929044f-1f43-4453-b2c0-a2f45dcdd4be,Sample Type / Medical Specialty: General Medi...,2014-09-23T00:00:00.000,M
4,9929044f-1f43-4453-b2c0-a2f45dcdd4be,Sample Type / Medical Specialty: General Medi...,2014-09-23T00:00:00.000,M


In [23]:
df.describe()

,subject_id,text,dob,gender
count,1044,1044,1044,1044
unique,94,141,79,2
top,3f98ac0f-83c0-4d90-b30f-d454a09a367b,Sample Type / Medical Specialty: General Medi...,1917-10-04T00:00:00.000,M
freq,42,21,70,607


## Save the CSV to file

In [24]:
df.to_csv("data/data.csv")

## Save the text also

In [25]:
f = open("data/data.txt", 'w')
for text in df['text'].values:
    f.write(text)
    f.write('\n')
f.close()